# Lattice library MWE

This notebook contains the minimum working examples for reading the OpenQCD Gen2L gauge field files and performing some basic operations.

In [1]:
import lyncs_io as io

import gluon_utils
import numpy as np

In [2]:
# Change these as appropriate
path_to_gaugefield = "/home/ben/Work/gauge_confs/confs/full/8x32/Gen2l_8x32n1"
path_to_transform = "/home/ben/Work/gauge_confs/transforms/landau/full/8x32/Gen2l_8x32n1.gauge.lime"

Nt = 8
pattern = "landau"  # Type of gauge transform.

# Don't change these
Ns = 32
Nd = 4
Nc = 3

### Read gauge field and transform files

In [3]:
gaugefield = io.load(path_to_gaugefield, format="openqcd")
transform = io.load(path_to_transform, format="lime")

In [4]:
print("Gauge field shape:", gaugefield.shape)
print("Transform shape:", transform.shape)

Gauge field shape: (8, 32, 32, 32, 4, 3, 3)
Transform shape: (8, 32, 32, 32, 3, 3)


## Create lattice object from gaugefield

In [5]:
lattice = gluon_utils.lattice(gaugefield,params=(Nt,Ns,Nd,Nc))

print("Lattice shape:",lattice.lattice.shape)

Lattice shape: (8, 32, 32, 32, 4, 3, 3)


## Evaluate plaquette

This still uses Python code to evaluate the average plaquette and is VERY SLOW. Uncomment if you want to test it for yourself...

In [6]:
%%time

#P_s, P_t = lattice.evaluate_average_plaquette()
#np.mean([P_s,P_t])

CPU times: user 1min 2s, sys: 65.2 ms, total: 1min 2s
Wall time: 1min 2s


1.6237173258078674

In [7]:
print("Stored value:",io.head(path_to_gaugefield, format="openqcd")["plaq"])

Stored value: 1.6237173258079076


## Evaluate gauge condition before and after gauge fixing.

This is extremely fast compared to the standard Pythonic approach due to the side-step into C. Here we want the divergence to be below 1 part in $10^{16}$.

In [8]:
%%time

print("Before fixing:", lattice.evaluate_divA(pattern=pattern))

lattice.apply_gauge(transform)

print("After fixing:", lattice.evaluate_divA(pattern=pattern))

Before fixing: 827.5540333141095
After fixing: 9.854228997379667e-17
CPU times: user 2.68 s, sys: 5 µs, total: 2.68 s
Wall time: 2.68 s


The `lattice` object now contains the prepared (gauge-fixed) configuration ready for propagator calculation.